In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cristinabaldor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/cristinabaldor/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#reading in json file of intents and responses

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

87 documents
16 classes ['about', 'animalstudies', 'complaint', 'dadjokes', 'goodbye', 'greeting', 'help', 'loginproblems', 'name', 'positivity', 'provostsawards', 'researchadmin', 'researchdevelopment', 'thanks', 'viceprovost', 'vprsgeneral']
110 unique lemmatized words ["'s", 'a', 'about', 'access', 'administration', 'am', 'an', 'and', 'animal', 'anyone', 'apply', 'are', 'award', 'bye', 'call', 'can', 'charge', 'complaint', 'could', 'discouraged', 'do', 'doing', 'done', 'encouragement', 'facility', 'fellowship', 'for', 'funding', 'get', 'give', 'going', 'good', 'goodbye', 'grant', 'hand', 'have', 'having', 'hay', 'hello', 'help', 'helpful', 'hey', 'hi', 'hola', 'i', 'in', 'inspiration', 'internal', 'is', 'it', 'joke', 'know', 'lab', 'later', 'laugh', 'lead', 'log', 'login', 'make', 'me', 'mouse', "n't", 'name', 'need', 'not', 'now', 'of', 'office', 'order', 'os', 'please', 'pra', 'problem', 'program', 'provost', 'raise', 'research', 'sad', 'scholarship', 'see', 'service', 'should', '

In [5]:


# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training,dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [6]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
18/18 [==============================] - 1s 1ms/step - loss: 2.7700 - accuracy: 0.1008
Epoch 2/200
18/18 [==============================] - 0s 1ms/step - loss: 2.6653 - accuracy: 0.1400
Epoch 3/200
18/18 [==============================] - 0s 1ms/step - loss: 2.4920 - accuracy: 0.2166
Epoch 4/200
18/18 [==============================] - 0s 1ms/step - loss: 2.4256 - accuracy: 0.3063
Epoch 5/200
18/18 [==============================] - 0s 1ms/step - loss: 2.2663 - accuracy: 0.3453
Epoch 6/200
18/18 [==============================] - 0s 1ms/step - loss: 2.0875 - accuracy: 0.4965
Epoch 7/200
18/18 [==============================] - 0s 1ms/step - loss: 1.7332 - accuracy: 0.6353
Epoch 8/200
18/18 [==============================] - 0s 1ms/step - loss: 1.6128 - accuracy: 0.6789
Epoch 9/200
18/18 [==============================] - 0s 2ms/step - loss: 1.2305 - accuracy: 0.7732
Epoch 10/200
18/18 [==============================] - 0s 2ms/step - loss: 0.8847 - accuracy: 0.8894
Epoch 11/

18/18 [==============================] - 0s 2ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 84/200
18/18 [==============================] - 0s 2ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 85/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 86/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 87/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 88/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 89/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 90/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 91/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 92/200
18/18 [==============================] - 0s 1ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 93/200
